### **Create delta lake dim_date table using SQL**

In [0]:
%sql
CREATE TABLE IF NOT EXISTS finance_cata.gold.dim_date
USING DELTA
LOCATION 'abfss://gold@stfinancedev.dfs.core.windows.net/dim_date'
AS
WITH DateSequence(
  SELECT explode(sequence(to_date('2010-01-01'),to_date('2030-12-31'),interval 1 day)) as FullDate
)
SELECT
    CAST(date_format(FullDate, 'yyyyMMdd') AS INT) AS DateKey,
    FullDate,
    YEAR(FullDate) AS Year,
    MONTH(FullDate) AS Month,
    date_format(FullDate, 'MMMM') AS MonthName,
    CASE WHEN QUARTER(FullDate) = 1 THEN 'Q1'
         WHEN QUARTER(FullDate) = 2 THEN 'Q2'
         WHEN QUARTER(FullDate) = 3 THEN 'Q3'
         WHEN QUARTER(FullDate) = 4 THEN 'Q4' END AS Quarter,
    weekofyear(FullDate) AS WeekOfYear,
    day(FullDate) AS DayOfMonth,
    dayofweek(FullDate) AS DayOfWeek,
    date_format(FullDate, 'EEEE') AS DayName,
    CASE WHEN dayofweek(FullDate) IN (1,7) THEN 'WeekEnd' ELSE 'WorkDay' END AS IsWeekend
FROM DateSequence
ORDER BY FullDate;

In [0]:
df = spark.sql("""select * from finance_cata.gold.dim_date""")

In [0]:
df.display()

In [0]:
df.write.format("delta").mode("overwrite").save('abfss://gold@stfinancedev.dfs.core.windows.net/dim_date')